# N-vezér probléma

## Bevezetés

Az $N$-vezér probléma egy sakkfeladvány, melynek lényege, hogy hányféleképpen tudunk $N$ vezért elhelyezni egy $N \times N$-es sakktáblán úgy, hogy azok a sakk szabályai szerint ne üssék egymást. A probléma érdekes és egyszerűnek tűnik, viszont nem triviális, és numerikus megoldása is nagy számítógépes erőforrásokat igényel - az első kilenc $N$-vezér probléma futtatása kevesebb, mint $10$ percet vett igénybe, de a tizedik már sokkal többet, mint az előzőek együtt.

## Megvalósítás

Előzetes műveletek címszó alá kerültek azon kódrészek, melyek technikai okokból elengedhetetlenek, de a sakkprobléma megoldásához közvetlen módon nem járulnak hozzá. Ezeket nem részletezném, csupán azért részei a notebook-nak, mert ezek nélkül nem futna le. (Vannak látszólag redundáns függvények is, melyek egyszer sincsenek meghívva, mankóként használtam őket.)

A következő szekció az $N$-vezér problémát ténylegesen megvalósító függvényeket tartalmazza. Minden matematikai művelet itt található. Ezek a függvények már részletezve vannak. A definíció feletti sorban található, hogy mi a függvény szerepe, és a függvénydefiníción belül (kommentek formájában) szerepelnek a fontos lépések magyarázatai.

A probléma megoldását implementáló függvény ("program(n)") először meghatározza adott $N$-re a különböző megoldásokat (nem csak a megoldások számát, a vezérek koordinátáit is), majd ezen megoldásokon végigfut, kivizsgálva a szimmetriákat (egy-egy megoldás tükrözöttje-e (bármelyik átlón és középvonalon keresztül) vagy elforgatottja-e a másiknak - és ezeknek az összes lehetséges lineárkombinációja) visszaadja a lényegesen különböző megoldásokat. Végül a különböző és a lényegesen különböző megoldások koordinátáit elmenti egy file-ba, számukat kiírja a cella alá. Először a "program(n)" optimalizált rekurzív magjáról lesz szó, mely visszaad egy rész-megoldás-halmazt. Ezután a rekurzív mag eredményeinek tisztításáról, a végeredményekig való eljutásról lesz szó.

### Előzetes műveletek

#### Szükséges könyvtárak importálása

In [1]:
import numpy as np
import scipy
import os
import matplotlib.pyplot as plt
from matplotlib import colors

#### File-kezeléshez szükséges függvények

In [2]:
def filein(name, n):
    a = []
    with open(name) as file:
        lines = file.readlines()
    for i in lines:
        a.append([])
        for j in range(n):
            a[-1].append(int(i[j]))
    return a
    
def fileout(a, name):
    s = ""
    n = len(a)
    for i in range(n):
        for j in range(n):
            s += str(a[i][j])
        s += "\n"
    with open(name, "w") as file:
        file.write(s)
        
def output(data, name):
    s = ""
    if(data!=[]):
        n = len(data[0])
        for i in data:
            for j in i:
                s += "[" + str(j[0]) + "," + str(j[1]) + "] "
            s += "\n"
    with open(name, "w") as file:
        file.write(s)

#### Adatstruktúrák kezeléséhez szükséges függvények

In [3]:
def cp(a):
    asd = []
    for i in a:
        asd.append(i.copy())
    return asd

def cp_extra(a):
    asd = []
    for i in a:
        asd.append(cp(i))
    
    return asd

### Az $N$-vezér problémát megoldó függvények

A sakktáblát egy $2D$ tömbbel (mátrixszal) reprezentáltam. Egy $M$ mátrix esetén $M[x]$ a sakktábla $(x+1)$-edik sora és $M[x][y]$ a sakktábla $(x+1)$-edik sorának $(y+1)$-edik eleme.

Kiinduló helyzetben a sakktáblát reprezentáló mátrix minden eleme "$1$". Azokba a cellákba, ahova vezért helyeztünk, "$2$"-es kerül. "$0$" kerül azokba a cellákba, melyeket bármelyik vezér üti.

$\textbf{Megjegyzés :}$ Zavaró lehet, hogy mindig $(n+1)$-edik vagy $(j-1)$-edik sorról vagy oszlopról van szó. Azért jön be a $\pm 1$, mert a Python $0$-tól indexel.

#### A rekurzív mag által használt függvények

Az alábbi két függvény a két leggyakrabban meghívott függvény. E két függvény szerepe, hogy egy sorban le tudjuk ellenőrizni, szabad-e adott cellába vezért helyezni (van-e ott vezér vagy ütve van-e a cella), illetve szintén egy sorban el tudjuk végezni a vezér-elhelyezés műveletét.

In [4]:
# egy NxN-es mátrixba bejelöl a megadott cellába egy vezért, és bejelöli az általa ütött cellákat
def gridn(L, pos):
    x = pos[0]
    y = pos[1]
    N = len(L) # hányszor hányas a sakktábla
    
    for i in range(N): # dupla ciklussal átláthatóbb a megoldás, de egy N-es faktorral lassabb, és ez sokat számíŧ
        L[x][i] = 0 # a vezér sorának nullázása
        L[i][y] = 0 # a vezér oszlopának nullázása
        # a diagonális és az antidiagonális nullázása
            # az alábbi (A) ellenőrzésekre azért van szükség, mert a (B) egyeneseknek csak a sakktáblára eső
            # részét kell figyelembe venni - különben túl is indexelné az L tömböt
        a = x+y-i # (B)
        if((a>-1)and(a<N)): # (A)
            L[i][a] = 0
        a = i-x+y # (B)
        if((a>-1)and(a<N)): # (A)
            L[i][a] = 0

# leellenőrzi, szabad-e az adott cellába vezért helyezni
def check(l, pos):
    x = pos[0]
    y = pos[1]
    res = 1
    if((l[x][y] == 0)or(l[x][y] == 2)): # "ha ütve van a cella vagy vezér van ott"
        res = 0
    return res

#### A rekurzió

Ez a függvény a megoldás motorja. Az "$l$" argumentum a sakktábla aktuális állását reprezentáló $2D$ tömb, a "$vec$" tömbben tárolódnak a részmegoldások - pl. az $5.$ vezér letételekor ebben a változóban kapja meg a függvény, hogy hol van az előző $4$ -, míg a "$data$" tömbhöz a megoldások vannak hozzáfűzve, és tárolva benne - egy sor tartalmazza az $n\times2$ koordinátát.

A rekurzióban meghívott rekurzió megkap egy másolatot az előző iteráció részmegoldásáról a "$vec$" és az $l$ tömb formájában. A "$data$" tömbről nem másolat készül - direkt módon ugyanazt a "$data$" tömböt módosítja a rekuzió minden iterációja, mint amit először megadtunk neki (az üres tömböt). Az előbbi azért jó, mert a másolat módosítva lesz, és a ciklus következő iterációjában ugyanazt a tömböt szerenénk használni. Az utóbbi azért praktikus, mert egy függvényhívás-gráfban globális változóként viselkedik.

Az "$l$" tömbben szerepel az az információ, hogy hányszor hányas sakktábláról van szó, a "$vec$" tömb alapján pedig meg tudjuk határozni, hogy hol tartunk a rekurziós fán (a hossza kifejezi, hogy hány darab vezért helyeztünk már el). A rekuzió kilépési feltétele ezek alapján egy $N\times N$-es sakktábla esetén az, hogy "$vec$" hossza eléri $N$-t. Ekkor a "$vec$" tömbben tárolt $n\times2$ koordináta hozzáfűződik a "$data$" tömbhöz.

Legyen a "$vec$" tömb hossza "$n$". Amennyiben $n<N$, egy ciklus leellenőrzi cellánként, hogy az $(n+1)$-edik sorban hol van szabad hely. Ha talál szabad cellát, elhelyez oda egy vezért. Ezután meghívja a függvény saját magát, immár tudva, hogy az $(n+1)$-edik sorban már van egy vezér, és az $(n+2)$-edik sorban kell keresni a következőt. Amennyiben nem talál szabad helyet a következő sorban, a "$vec$" tömb automatikusan törlődik, és a rekurzió előző iterációjában lévő ciklus következő iterációjára ugrunk - lényegében ez történik akkor is, amikor találunk megoldást, csak ekkor már végigmentünk az összes soron és elmentjük a "$vec$" tartalmát.

In [5]:
def recn(l, vec, data):
    
    N = len(l) # hányszor hányas a sakktábla
    n = len(vec) # mennyi vezér van lehelyezve = ahanyadik sorindexnél keressük a megoldást
    lc = cp(l) # csinálunk egy biztonsági (de szükséges) mentést a tömbről, hogy az iterációk alatt ne változzon
    if(n<N): # "ha még nem raktunk vezért az utolsó sorba"
        for j in range(N): # az (n-1)-edik sor celláin végigmegyünk
            vecc = cp(vec) # hasonló biztonsági mentés
            lcc = cp(lc) # hasonló biztonsági mentés
            pos = [n, j] # az (n-1)-edik sor (j+1)-edik celláját vizsgáljuk
            if(check(lcc, pos)==1): # "ha szabad a cella"
                gridn(lcc, pos) # elhelyezzük a vezért
                vecc.append(pos) # hozzáfűzzük a részmegoldáshoz az imént elhelyezett vezér koordinátáit
                recn(lcc, vecc, data)
    if(n==N): # kilépési feltétel, "ha annyi vezért elhelyeztünk, amennyi sora van a sakktáblának"
        data.append(vec) # elmentjük a jó megoldást

#### Sorrend ellenőrzés

A program tesztelése során sokszor működött rosszul a rekurzió - és végül egyszer jól. Ennek kivizsgálása céljából írtam az alábbi két függvényt. Összességében az alsó függvény - a felső segítségével - meghatározza, hogy visszakaptuk-e többször ugyanazt a megoldást oly módon, hogy egy-egy megoldásban ugyazokba a cellákba kerültek vezérek, csak más sorrendben. A rekurzióban használt matematikai modell nem engedi meg, hogy kétszer visszakapjuk ugyanazt a megoldást, de ezzel az ellenőrzéssel kicsi erőforrással meggyőződhetünk arról, hogy legalább ebben biztosan követi a matematikai modellt az implementációja is.

In [6]:
# ez a függvény egy-egy megoldásról eldönti, hogy megegyeznek-e a fent említett módon
def dup(a, b):
    f = False
    res = 0
    n = len(a)
    for i in a:
        for j in b:
            if(i==j):
                res += 1
    if(res==n):
        f = True
    return f

# egy egész megoldáshalmazon végigmegyünk, és kidobáljuk az egyező megoldásokat
def duplicate(data):
    if(data==[]):
        s = []
    else:
        s = [data[0]]
        
        for i in data:
            buff = 0
            for j in s:
                if(dup(i,j)):
                    buff+=1
            if(buff==0):
                s.append(i)
    return s


# a függvények ellenőrzése
dat = [[[1, 2],[3, 4]],[[3, 4],[1, 3]],[[1, 2],[3, 4]]]
duplicate(dat)

[[[1, 2], [3, 4]], [[3, 4], [1, 3]]]

#### Mátrix-Koordinátasor Konverzió

A fentiekben már előfordult a két használt adatstruktúra.

Az első egy $N\times N$-es mátrix, melyben tároljuk az összes cellát. Ez azért praktikus, mert könnyen bejelölhető rajta, hogy melyik cellának milyen státusza van, és a forgatások / tükrözések kivitelezhetőbbek vele - az utóbbi miatt szükséges a konverzió függvények definiálása.

A második adatstruktúra egy tömb, melyben csak a vezérek celláinak koordinátái vannak tárolva. Ez praktikus a megoldások tárolására, mert nem tárolunk el feleslegesen sok "$0$"-t. Másfelől ez a negyedik verziója a notebook-nak, és az első kettő alatt megtelt a RAM memória, ezért döntöttem egy kompaktabb megoldás-tárolás mellett.

In [7]:
# vezér-koordinátákból mátrixba
def pos_to_mat(pos):
    n = len(pos)
    l = [] # létrehozzuk a mátrixot és feltöltjük "1"-esekkel
    for i in range(n):
        l.append([])
        for j in range(n):
            l[-1].append(1)
    for i in pos: # ahol van vezér, oda "2"-est teszünk
        l[i[0]][i[1]] = 2
    return l


# mátrixból vezér-koordinátákba
def mat_to_pos(l):
    n = len(l)
    res = [] # ide jönnek a vezérek koordinátái
    for i in range(n):
        for j in range(n):
            if(l[i][j]==2): # "ha az [i][j] cellában vezér van"
                res.append([i, j]) # hozzáfűzzük a vezér koordinátáit a tömbhöz
    return res

#### Forgatások és tükrözések

A "Nyolckirálynő-probléma" Wikipédia cikkben a "különböző megoldások" megegyezik a rekuzió végereményére hattatott "duplicate()" visszatérési értékével. A "lényegesen különböző megoldások" azok, amelyek nem tükrözöttjei és/vagy elforgatottjai egymásnak. Ebben a szekcióban szerepelnek az elemi forgatásokat és tükrözéseket megvalósító függvények, majd az ezek összes lineárkombinációját kiszámoló függvény, és végül az a függvény, mely az előbbiek segítségével egy egész megoldáshalmazból kiválogatja a lényegesen különböző megoldásokat.

In [8]:
# elemi forgatást és tükrözést megvalósító függvények

# elemi forgatás (90 fokkal)
def rot_elem(l0):
    n = len(l0)
    l = pos_to_mat(l0)
    lc = cp(l)
    for i in range(n):
        for j in range(n):
            lc[j][n-1-i] = l[i][j] # koordináta-transzformáció
    return mat_to_pos(lc)

# tükrözés horizontális tengelyen
def mirror_hor_elem(l0):
    n = len(l0)
    l = pos_to_mat(l0)
    lc = cp(l)
    for i in range(n):
        for j in range(n):
            lc[n-1-i][j] = l[i][j] # koordináta-transzformáció
    return mat_to_pos(lc)

# tükrözés vertikális tengelyen
def mirror_vert_elem(l0):
    n = len(l0)
    l = pos_to_mat(l0)
    lc = cp(l)
    for i in range(n):
        for j in range(n):
            lc[i][n-1-j] = l[i][j] # koordináta-transzformáció
    return mat_to_pos(lc)

# tükrözés diagonális tengelyen
def mirror_diag_elem(l0):
    n = len(l0)
    l = pos_to_mat(l0)
    lc = cp(l)
    for i in range(n):
        for j in range(n):
            lc[j][i] = l[i][j] # koordináta-transzformáció
    return mat_to_pos(lc)

# tükrözés antidiagonális tengelyen
def mirror_antidiag_elem(l0):
    n = len(l0)
    l = pos_to_mat(l0)
    lc = cp(l)
    for i in range(n):
        for j in range(n):
            lc[n-1-j][n-1-i] = l[i][j] # koordináta-transzformáció
    return mat_to_pos(lc)

In [9]:
# a tükrözés és az elforgatás összes lineárkombinációját kiszámoló függvény
    # a fentit kiszámolja "a"-ra, majd leellenőrzi, "b" benne van-e ebben a halmazban
def rm(a, b):
    f = True # ha nem egybevágóak a megoldások, True értéket ad vissza a függvény
    res = 0
    n = len(a)
    dat = [cp(a)]
    for i in range(3):
        datc = rot_elem(cp(dat[-1]))
        dat.append(datc)
        dat.append(mirror_hor_elem(datc))
        dat.append(mirror_vert_elem(datc))
        dat.append(mirror_diag_elem(datc))
        dat.append(mirror_antidiag_elem(datc))
    # meglévő függvények segítségével leellenőrizzük, hogy az "a" megoldás egybevágó-e a "b"-vel
    bs = duplicate(dat) # kiszedjük a megegyező megoldásokat a lineárkombinációk halmazából
    bs.append(b) # hozzáfűzzük a "b" tömböt
    s = duplicate(bs) # kiszedjük a megegyező megoldásokat
    if(len(s)==len(bs)): # ha ezen két tömb hossza megegyezik, "b" egybevágó volt "a"-val
        f = False
    return f

# kiválogatja a megoldáshalmazból a lényegesen különböző megoldásokat
def rotmir(data):
    if(data==[]):
        s = []
    else:
        s = [data[0]] # a lényegesen különböző megoldások tömbje
        for i in data: # végigmegyünk az összes megoldáson
            buff = 0
            for j in s: # ha nem egybevágó a megoldás egy lényegesen különböző megoldással sem,*
                if(rm(i,j)):
                    buff+=1
            if(buff==0):
                s.append(i) #* bekerül a lényegesen különböző megoldások halmazába
    return s

## Eredmények

A kompaktság kedvéért két utolsó függvényt definiáltam. A "vezer(n)" függvénynek csak azt kell megadni, hogy a hány-vezér-problémát szeretnénk megoldani - ez alapján kigenerál egy kezdőállapotú sakktáblát és a rekurzív függvény segítségével visszaadja a megoldáshalmazt. A "program(n)" tartalmazza az előző függvény meghívását is, ezen végez duplicate-ellenőrzést, egybevágóság-ellenőrzést, kiírja a képernyőre a megoldások számát, valamint elmenti a megoldásokat 2-2 (különböző és lényegesen különböző megoldáshalmazokat külön-külön) szöveges file-ba.

In [10]:
def vezer(n):
    proba = []
    for i in range(n):
        proba.append([])
        for j in range(n):
            proba[-1].append(1)
    data = []
    recn(proba, [], data)
    
    return data

In [11]:
def program(n):
    res = vezer(n)
    res0 = duplicate(res)
    if(len(res0)==len(res)):
        print("No duplicates found")
    else:
        print("Duplicates found and eliminated")
    result = rotmir(res)
    
    output(res, "vezer" + str(n) + "_kulonbozo.txt")
    output(result, "vezer" + str(n) + "_lenyegesen_kulonbozo.txt")
    
    print(str(n) + " vezér probléma")
    print("\t- különböző megoldásainak száma : " + str(len(res)))
    print("\t- lényegesen különböző megoldásainak száma : " + str(len(result)))

### Futtatás és ellenőrzés

In [12]:
program(1)

No duplicates found
1 vezér probléma
	- különböző megoldásainak száma : 1
	- lényegesen különböző megoldásainak száma : 1


In [13]:
program(2)

No duplicates found
2 vezér probléma
	- különböző megoldásainak száma : 0
	- lényegesen különböző megoldásainak száma : 0


In [14]:
program(3)

No duplicates found
3 vezér probléma
	- különböző megoldásainak száma : 0
	- lényegesen különböző megoldásainak száma : 0


In [15]:
program(4)

No duplicates found
4 vezér probléma
	- különböző megoldásainak száma : 2
	- lényegesen különböző megoldásainak száma : 1


In [16]:
program(5)

No duplicates found
5 vezér probléma
	- különböző megoldásainak száma : 10
	- lényegesen különböző megoldásainak száma : 2


In [17]:
program(6)

No duplicates found
6 vezér probléma
	- különböző megoldásainak száma : 4
	- lényegesen különböző megoldásainak száma : 1


In [18]:
program(7)

No duplicates found
7 vezér probléma
	- különböző megoldásainak száma : 40
	- lényegesen különböző megoldásainak száma : 6


In [19]:
program(8)

No duplicates found
8 vezér probléma
	- különböző megoldásainak száma : 92
	- lényegesen különböző megoldásainak száma : 12


In [20]:
program(9)

No duplicates found
9 vezér probléma
	- különböző megoldásainak száma : 352
	- lényegesen különböző megoldásainak száma : 46


A Wikipédián szereplő megoldásokhoz képest nincs eltérés, sikeresnek mondható a megvalósítás.

### Utószó - avagy mire nem jutott idő

A probléma először triviálisnak tűnt, meglepő volt meggyőződni az ellenkezőjéről. Örülök, hogy végül sikerült egy elég gyorsan futó, memória-kímélő megvalósításra jutni.

Szerettem volna $N$ függvényében ábrázolni a futási időt. Érdekes lett volna meghatározni a program hatékonyságát.

A megoldások manuálisan ellenőrizhetőek, de egy ábrázoló függvényt is szerettem volna készíteni, ami kirajzolja a sakktáblát és rá a vezéreket.